# Import Libraries

In [3]:
import pandas as pd
import numpy as np
import re
import sys
import math
from nltk.stem.porter import PorterStemmer
from python_utils import *

# Data Visualization

In [4]:
df = pd.read_csv(r"full-corpus.csv")

In [5]:
df.head()

,Topic,Sentiment,TweetId,TweetDate,TweetText
0,apple,positive,126415614616154112,Tue Oct 18 21:53:25 +0000 2011,Now all @Apple has to do is get swype on the i...
1,apple,positive,126404574230740992,Tue Oct 18 21:09:33 +0000 2011,@Apple will be adding more carrier support to ...
2,apple,positive,126402758403305474,Tue Oct 18 21:02:20 +0000 2011,Hilarious @youtube video - guy does a duet wit...
3,apple,positive,126397179614068736,Tue Oct 18 20:40:10 +0000 2011,@RIM you made it too easy for me to switch to ...
4,apple,positive,126395626979196928,Tue Oct 18 20:34:00 +0000 2011,I just realized that the reason I got into twi...


In [6]:
df.shape

(5113, 5)

In [7]:
df = df.drop(['Topic'],axis=1)

In [8]:
df = df.drop(['TweetDate'],axis=1)

In [9]:
df = df[["TweetId","Sentiment","TweetText"]]

In [10]:
d = {'negative': 0, 'positive': 1}
df['Sentiment'] = df['Sentiment'].map(d).fillna(df['Sentiment'])

In [11]:
df = df[df['Sentiment'] != 'neutral']
df = df[df['Sentiment'] != 'irrelevant']

In [65]:
df.head(5)

,TweetId,Sentiment,TweetText
0,126415614616154112,1,Now all @Apple has to do is get swype on the i...
1,126404574230740992,1,@Apple will be adding more carrier support to ...
2,126402758403305474,1,Hilarious @youtube video - guy does a duet wit...
3,126397179614068736,1,@RIM you made it too easy for me to switch to ...
4,126395626979196928,1,I just realized that the reason I got into twi...


In [13]:
df['TweetText'][27]

"When you want something done right, you do it yourself... or go to @Apple. AT&T you're useless these days. #yourdaysarenumbered\n\n\uf8ff\uf8ff\uf8ff"

In [14]:
df.shape

(1091, 3)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1091 entries, 0 to 3951
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   TweetId    1091 non-null   int64 
 1   Sentiment  1091 non-null   object
 2   TweetText  1091 non-null   object
dtypes: int64(1), object(2)
memory usage: 74.1+ KB


In [16]:
df.to_csv('new_corpus.csv', index=False)

# Data preprocessing

In [17]:
def preprocess_word(word):
    # Remove punctuation
    word = word.strip('\'"?!,.():;')
    # Convert more than 2 letter repetitions to 2 letter
    # funnnnny --> funny
    word = re.sub(r'(.)\1+', r'\1\1', word)
    # Remove - & '
    word = re.sub(r'(-|\')', '', word)
    return word

In [18]:
def is_valid_word(word):
    # Check if word begins with an alphabet
    return (re.search(r'^[a-zA-Z][a-z0-9A-Z\._]*$', word) is not None)

In [19]:
def handle_emojis(tweet):
    # Smile -- :), : ), :-), (:, ( :, (-:, :')
    tweet = re.sub(r'(:\s?\)|:-\)|\(\s?:|\(-:|:\'\))', ' EMO_POS ', tweet)
    # Laugh -- :D, : D, :-D, xD, x-D, XD, X-D
    tweet = re.sub(r'(:\s?D|:-D|x-?D|X-?D)', ' EMO_POS ', tweet)
    # Love -- <3, :*
    tweet = re.sub(r'(<3|:\*)', ' EMO_POS ', tweet)
    # Wink -- ;-), ;), ;-D, ;D, (;,  (-;
    tweet = re.sub(r'(;-?\)|;-?D|\(-?;)', ' EMO_POS ', tweet)
    # Sad -- :-(, : (, :(, ):, )-:
    tweet = re.sub(r'(:\s?\(|:-\(|\)\s?:|\)-:)', ' EMO_NEG ', tweet)
    # Cry -- :,(, :'(, :"(
    tweet = re.sub(r'(:,\(|:\'\(|:"\()', ' EMO_NEG ', tweet)
    return tweet

In [20]:
def preprocess_tweet(tweet):
    processed_tweet = []
    # Convert to lower case
    tweet = tweet.lower()
    # Replaces URLs with the word URL
    tweet = re.sub(r'((www\.[\S]+)|(https?://[\S]+))', ' URL ', tweet)
    # Replace @handle with the word USER_MENTION
    tweet = re.sub(r'@[\S]+', 'USER_MENTION', tweet)
    # Replaces #hashtag with hashtag
    tweet = re.sub(r'#(\S+)', r' \1 ', tweet)
    # Remove RT (retweet)
    tweet = re.sub(r'\brt\b', '', tweet)
    # Replace 2+ dots with space
    tweet = re.sub(r'\.{2,}', ' ', tweet)
    # Strip space, " and ' from tweet
    tweet = tweet.strip(' "\'')
    # Replace emojis with either EMO_POS or EMO_NEG
    tweet = handle_emojis(tweet)
    # Replace multiple spaces with a single space
    tweet = re.sub(r'\s+', ' ', tweet)
    words = tweet.split()

    for word in words:
        word = preprocess_word(word)
        if is_valid_word(word):
            if use_stemmer:
                word = str(porter_stemmer.stem(word))
            processed_tweet.append(word)

    return ' '.join(processed_tweet)

In [21]:
def preprocess_csv(csv_file_name, processed_file_name, test_file=False):
    save_to_file = open(processed_file_name, 'w')
    
    with open(csv_file_name, 'r',  encoding="utf8") as csv:
        lines = csv.readlines()
        total = len(lines)
        print(total)
        
        for i,line in enumerate(lines):
            #print()
            print(i," -> ",line)
            tweet_id = line[:line.find(',')]
            if tweet_id == "TweetId" or len(tweet_id) == 0 or tweet_id[0] != '1':
                continue
            if not test_file:
                line = line[1 + line.find(','):]
                #print(line[:line.find(',')]," ",type(line[:line.find(',')]))
                positive = int(float(line[:line.find(',')]))
            line = line[1 + line.find(','):]
            tweet = line
            processed_tweet = preprocess_tweet(tweet)
            
            #print(tweet_id," ",positive)
            
            if not test_file:
                save_to_file.write('%s,%d,%s\n' %
                                   (tweet_id, positive, processed_tweet))
            else:
                save_to_file.write('%s,%s\n' %
                                   (tweet_id, processed_tweet))
            #write_status(i + 1, total)
    save_to_file.close()
    print("Saved processed tweets to: %s" % processed_file_name)
    return processed_file_name

In [22]:
use_stemmer = False
csv_file_name = 'new_corpus.csv'
processed_file_name = 'new_corpus.csv'[:-4] + '-processed.csv'
if use_stemmer:
    porter_stemmer = PorterStemmer()
    processed_file_name = 'new_corpus.csv'[:-4] + '-processed-stemmed.csv'

print(csv_file_name)
print(processed_file_name)
print()
preprocess_csv(csv_file_name, processed_file_name, test_file=False)

new_corpus.csv
new_corpus-processed.csv

1130
0  ->  TweetId,Sentiment,TweetText

1  ->  126415614616154112,1.0,Now all @Apple has to do is get swype on the iphone and it will be crack. Iphone that is

2  ->  126404574230740992,1.0,@Apple will be adding more carrier support to the iPhone 4S (just announced)

3  ->  126402758403305474,1.0,Hilarious @youtube video - guy does a duet with @apple 's Siri. Pretty much sums up the love affair! http://t.co/8ExbnQjY

4  ->  126397179614068736,1.0,@RIM you made it too easy for me to switch to @Apple iPhone. See ya!

5  ->  126395626979196928,1.0,I just realized that the reason I got into twitter was ios5 thanks @apple

6  ->  126394830791254016,1.0,"I'm a current @Blackberry user, little bit disappointed with it! Should I move to @Android or @Apple @iphone"

7  ->  126379685453119488,1.0,The 16 strangest things Siri has said so far. I am SOOO glad that @Apple gave Siri a sense of humor! http://t.co/TWAeUDBp via @HappyPlace

8  ->  12637765641661

235  ->  126292335540699136,0.0,"@Apple iTunes is the worst program ever. For such a great phone, you make some awful software."

236  ->  126286814578348032,0.0,One would think the voice recognition on the @apple tech support line would work a little better.

237  ->  126283602571964416,0.0,"@apple $319 to repair my iPad 2, Apple you've lost me and my $700 a year, Android here I come!"

238  ->  126281019476291585,0.0,"@azee1v1 @apple @umber Proper consolidation, proper syncing, stop losing my PURCHASED items, checkboxes that do what you think they will do."

239  ->  126280555980529664,0.0,@AsimRang @apple @umber the desktop app is wack though

240  ->  126279811151831042,0.0,I made a reservation and yet I still have to wait in line. Humpt! Oh @apple (@ Apple Store w/ 2 others) http://t.co/JZmVBdNm

241  ->  126258214412091392,0.0,Frustrated that I bought a new macbook pro from @apple only to find it doesnt ship with media! #expensivepaperweight

242  ->  126257645282799616,0.0,Dear 

462  ->  125333948556521472,0.0,"…and only the first 23 images in my Photo Roll made it over? Seriously, @apple, how did you fuck this up so much?"

463  ->  125330038248849408,0.0,@apple why does #iPhone4 plus #iOS5 equal half the battery life?

464  ->  125329867674886144,0.0,@apple why isn't my iTunes updating? I just wanna update my iPhone to IOS5!!! Then iTunes says that it needs to update to do it. Then fails!

465  ->  125327896066785280,0.0,Now @apple won't let me use credit on @iTunes unless I have a credit card. Should have told me when I redeemed the cards. Sound fair to you?

466  ->  125313086465904641,0.0,"@amandbrar @apple Siri has worked for me less than 10% of the time. If service doesn't improve by Monday, I'm returning my iPhone 4S."

467  ->  125311989751877632,0.0,"@azrael: RT @DavideGori: And again #MobileMe is down. Issues when syncing, accessing mail... is this a good premise for #iCloud? @Apple, ..."

468  ->  125309975881977857,0.0,need @apple help!! installin

530  ->  126523731710443521,1.0,RT @shagorikah: Telegraph reports that the biggest threat to #facebook is its power users have shifted to #twitter or #google+ http://t. ...

531  ->  126523530903953408,1.0,Sorry #Apple #Google and #Samsung just made you look bad. #Android is king

532  ->  126523525598162944,1.0,Telegraph reports that the biggest threat to #facebook is its power users have shifted to #twitter or #google+ http://t.co/JHdTYKCV

533  ->  126522990585315328,1.0,#Facebook power users 'have gone to #Google+ and #Twitter' - Telegraph http://t.co/vwCiOv1f Who are they? #socialmedia

534  ->  126522810821644288,1.0,"Not impressed much with the new Android update. But good signs: a readable font, emphasis on design, and less nerdiness. #google"

535  ->  126522714713370624,1.0,"Video: #Google wallet, see the wow effect http://t.co/mcW78VtG"

536  ->  126522621251682304,1.0,When I'm about to tweet something and I cant remember how to spell I go straight to #Google

537  ->  12652

699  ->  126487090723700737,1.0,"As an iPhone user, I think the Samsung Nexus Galaxy looks great. And Ice Cream Sandwich looks fantastic. http://t.co/Z9IDoNi7 #Google"

700  ->  126487082905518081,1.0,WHAT!!!!!! #Google #Chrome syncs with your #Android device's bookmarks!?!?! NICE!

701  ->  126486849706401792,1.0,The Samsung Galaxy Nexus and Ice Cream Sandwich are sick! #android #icecreamsandwich #google

702  ->  126486332104130561,1.0,Inline keyboard spell check is nice #ICS with native voice dictation. #Android #Google

703  ->  126486322675318784,1.0,I can't wait to get the new #Galaxy #Nexus with #IceCreamSandwich... best yet it supports my #Google Wallet http://t.co/bqMuHxkK via @cnet

704  ->  126485171280166913,1.0,ICS on the Galaxy Nexus has very smooth and responsive screen and widget navigation. #Google #Android #ICS

705  ->  126484977176158208,1.0,New Galaxy Nexus: OS Improvements -  Built on Ice Cream Sandwich version of Android #nexus #samsung #google #android bit.ly/nE

780  ->  126750973329817601,1.0,Mango shows #Microsoft still has the taste for smartphone success http://t.co/AFm8VSMm  #wp7 #mango

781  ->  126749591956762624,1.0,RT @bmann Awesome! @TommyLee is moving to #Vancouver as Dev Evangelist for #Microsoft - we finally get a local

782  ->  126748497096622080,1.0,Awesome! @TommyLee is moving to #Vancouver as Dev Evangelist for #Microsoft - we finally get a local

783  ->  126748156003221504,1.0,Microsoft Stores offer up free Windows Phone 7 devices #microsoft #microsoftstores: Microsoft's retail stores ar... http://t.co/r2RndN6Y

784  ->  126748155021762561,1.0,Microsoft Stores offer up free Windows Phone 7 devices #microsoft #microsoftstores http://t.co/xoLRaueY #microsoft

785  ->  126748153952206849,1.0,Microsoft Stores offer up free Windows Phone 7 devices #microsoft #microsoftstores http://t.co/F4HwAN6Q

786  ->  126747965393084416,1.0,Microsoft Stores offer up free Windows Phone 7 devices #microsoft #microsoftstores http://t.co/7YvJMHn

912  ->  126700014385897472,0.0,"I sware to go, #windows and #microsoft will be the death of me...I don't know why I keep giving windows (cont) http://t.co/3rkOchzr"

913  ->  126692062757269505,0.0,RT @EA_Jorge: I'm on day 80-something of my Xbox LIVE acct being hacked. This is worse than PSN hack. Will I ever get it back? #Microsof ...

914  ->  126691815394000896,0.0,Thanks again #microsoft for wasting more of my time. Apple&gt;windows http://t.co/GNy9G07B

915  ->  126690175656001538,0.0,"Dear #Microsoft, I should only have to hit the Windows update button *once* when manually running updates, not 4 or 5 times. #sysadmin"

916  ->  126689051960033280,0.0,"Totally unproductive day with no access to #Microsoft Excel or Word, aaargh! Will be working late tonight after hubby fixes. #fingerscrossed"

917  ->  126686283769712640,0.0,RT @ManelFMartinez: How #Apple eclipsed #Microsoft http://t.co/IJ6XyDp3  #sun rise #in via @guardian

918  ->  126686075287642112,0.0,How #Apple eclipsed #Mi

979  ->  126543225501650944,0.0,#Apple eclipsed #Microsoft http://t.co/wEdPbcPx

980  ->  126525932625334272,0.0,http://t.co/5oK4XcOw Microsoft !! i hate you again !! #microsoft

981  ->  126519630868119552,0.0,"Open Letter: Steve Ballmer, Please Retire - Forbes - Please retweet! #MSFT needs a techie leader. http://t.co/E8uMRmro #microsoft #ballmer"

982  ->  126518620095393792,0.0,@MatthewGPhy One Operating System to rule them all and in the darkness bind them.  #Microsoft

983  ->  126517164416049152,0.0,"I wonder if I tweet Bill Gates he'll fix my laptop... 

984  ->  

985  ->  #Microsoft

986  ->  #vista

987  ->  #fml"

988  ->  126515911321591808,0.0,RT @anup_pillai: How #Apple eclipsed #Microsoft | http://t.co/r6hNEtTu

989  ->  126513363529383937,0.0,"After I don't know how many years, you still can't copy a #microsoft #word document headings to #powerpoint. #justsaying"

990  ->  126508179281547264,0.0,"#microsoft #careers site is giving errors for any thing i search!! #weird

Saved processed tweets to: new_corpus-processed.csv


'new_corpus-processed.csv'

In [45]:
with open('new_corpus-processed.csv', 'r',  encoding="utf8") as csv:
        lines = csv.readlines()
        
        for i,line in enumerate(lines):
            print(i," -> ",line)

0  ->  126415614616154112,1,now all USER_MENTION has to do is get swype on the iphone and it will be crack iphone that is

1  ->  126404574230740992,1,USER_MENTION will be adding more carrier support to the iphone just announced

2  ->  126402758403305474,1,hilarious USER_MENTION video guy does a duet with USER_MENTION s siri pretty much sums up the love affair URL

3  ->  126397179614068736,1,USER_MENTION you made it too easy for me to switch to USER_MENTION iphone see ya

4  ->  126395626979196928,1,i just realized that the reason i got into twitter was ios5 thanks USER_MENTION

5  ->  126394830791254016,1,im a current USER_MENTION user little bit disappointed with it should i move to USER_MENTION or USER_MENTION USER_MENTION

6  ->  126379685453119488,1,the strangest things siri has said so far i am soo glad that USER_MENTION gave siri a sense of humor URL via USER_MENTION

7  ->  126377656416612353,1,great up close personal event USER_MENTION tonight in regent st store

8  ->  1263

741  ->  126788430679113728,1,buffalo intel microsoft ibm are the best pavilions at smau

742  ->  126780006964805632,1,windows phone mango update process ahead of schedule URL microsoft windowsphone mango

743  ->  126779403605770241,1,wp7 is getting there microsoft claws back to smartphone relevance rich jaroslovsky bloomberg URL via USER_MENTION

744  ->  126779217911349248,1,on microsoft excel making functions like its my job productive

745  ->  126774092274741248,1,USER_MENTION why i love microsoft microsoft hosting kids techfest to encourage careers in tech URL babygeeks

746  ->  126768366345138176,1,microsoft word works on my computer again yess

747  ->  126767508253454336,1,in terms of unified communications giants really surprised to see how underrated microsoft is

748  ->  126762337087655936,1,microsoft dishing out free firstgen wp7 handsets at its stores URL

749  ->  126755785391869954,1,will watch this one with interest USER_MENTION microsoft codename data explorer ctp

In [52]:
processed_data = pd.read_csv(r"new_corpus-processed.csv", header=None)

In [53]:
processed_data.head()

,0,1,2
0,126415614616154112,1,now all USER_MENTION has to do is get swype on...
1,126404574230740992,1,USER_MENTION will be adding more carrier suppo...
2,126402758403305474,1,hilarious USER_MENTION video guy does a duet w...
3,126397179614068736,1,USER_MENTION you made it too easy for me to sw...
4,126395626979196928,1,i just realized that the reason i got into twi...


# Splitting into training and testing data

In [66]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(processed_data, test_size=0.2, random_state=42, shuffle=True)

In [67]:
# save the train and test file
# again using the '\t' separator to create tab-separated-values files
train.to_csv('train-processed.csv', sep=',', index=False)
test.to_csv('test-processed.csv', sep=',', index=False)

In [69]:
checking_train = pd.read_csv(r'train-processed.csv')

checking_train.head()

,0,1,2
0,126610651916410881,1,yahoo up sale again yrs back microsoft almost ...
1,126789739180326913,0,update microsoft ms11078 critical vulnerabilit...
2,126637450008346624,0,nokia employees still worried that elop is a m...
3,125956403574747137,0,whats wrong with siri today USER_MENTION
4,126513620686352384,1,ice cream sandwich u r really delicious can di...


In [70]:
checking_test = pd.read_csv(r'test-processed.csv')

checking_test.head()

,0,1,2
0,126750481069510656,0,microsoft is currently paying to produce swpat...
1,126570180070481920,0,USER_MENTION stand up for your freedom to inst...
2,126861382078578688,0,twitter still not showin my mothafuckin retweets
3,126863821594832897,1,dear twitter i have missed you promise ill kee...
4,126747960900984832,1,microsoft stores offer up free windows phone d...


In [71]:
print(checking_train.shape)
print(checking_test.shape)

(872, 3)
(219, 3)
